# Molecular diffusion study

In [ ]:
import sys
sys.path.append('../polysim')
import builder
import numpy
from matplotlib import pyplot

### Build a system containing 100 molecules each containing 100 beads.

In [ ]:
molecules = 100
molecule_size = 100
system = builder.construct_random_chains(molecules=molecules, molecule_size=molecule_size)
system.write_lammps_data('polymer.lammps')

In [ ]:
! mpiexec -n ${SLURM_CPUS_PER_TASK} --oversubscribe lmp_mpi -in in.lammps

### Reads the atomic trajectory file (dump file)

In [ ]:
fid = open('dump.lammpstrj')
steps = []
for line in fid:
    if 'ITEM: TIMESTEP' in line:
        timestep = int(next(fid))
    elif 'ITEM: NUMBER OF ATOMS' in line:
        n = int(next(fid))
    elif 'ITEM: ATOMS' in line:
        atom_data = numpy.loadtxt(fid, max_rows = n)
        atom_data = atom_data[numpy.argsort(atom_data[:,0])]
        steps.append(atom_data)

time = []
msd = []
for i in range(1, len(steps)):
    time.append(i*0.005 * 50)
    msd.append(0.0)
    for m in range(molecules):
        i0 = m * molecule_size
        i1 = (m+1) * molecule_size

        com = numpy.mean(steps[i][i0:i1, -3:], axis=0)
        com0 = numpy.mean(steps[0][i0:i1, -3:], axis=0)
        
        msd[-1] += numpy.sum((com - com0)**2)
    msd[-1] /= molecules

In [ ]:
# Plots MSD vs time on a log-log plot.
fig, ax = pyplot.subplots()
ax.loglog(time, msd, '.')
ax.set_xlabel('Time')
ax.set_ylabel('MSD')
slope = numpy.polyfit(numpy.log(time[5:]), numpy.log(msd[5:]), 1)[[0]]

# See https://en.wikipedia.org/wiki/Mean_squared_displacement
D = msd[-1] / (6*time[-1])
print(f'Diffusion rate is {D:.3f}')